In [23]:
import sys
import pandas as pd
import pickle
sys.path.append('../twc_api/')
import boto3
import json
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
session = boto3.Session(profile_name='twc')
s3 = session.resource('s3')

In [9]:
from api.utils.transformers import *
from api.utils.aws import *

In [10]:
from scipy.stats import spearmanr
from sklearn.ensemble import ExtraTreesRegressor

In [11]:
ConsolidateTablesTransformer(count_encode=False)

# First recreate the model we've been using to date

### Generate Training Data

In [27]:
generator = TrainingDataGenerator('../../Welcome-Centre-DataCorps-Data/ClientDatabaseStructure.mdb.sqlite')
training_data = generator.get_training_data(limit=100000)

In [28]:
json_tables = {k: v.to_json(orient='records') for k, v in training_data.items()}

### Build your Transformer and Choose Features

In [29]:
features_to_split = []

transformer = TransformerPipeline([
                        ConsolidateTablesTransformer(count_encode=False),
                        AddFutureReferralTargetFeatures(),
                        TimeFeatureTransformer(break_length=28),
                        SplitCurrentAndEverTransformer(['referralissue_', 
                                                       'referraldomesticcircumstances_',
                                                        'referralreason_', 'referralbenefit_'])
                                    ], aligner=AlignFeaturesToColumnSchemaTransformer())

X, y, referral_table = transformer.fit_transform(training_data)

X = X.fillna(0)

In [30]:
X_train = X[X.index < 13500]
X_test = X[X.index >= 13500]
y_train = y.loc[X_train.index]
y_test = y.loc[X_test.index]

# Build a model

In [31]:
from sklearn.ensemble import ExtraTreesRegressor

In [32]:
et = ExtraTreesRegressor(n_jobs=-1, n_estimators=500)
et.fit(X_train, y_train)

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
          oob_score=False, random_state=None, verbose=0, warm_start=False)

# Evaluate model

In [33]:
y_pred = pd.Series(et.predict(X_test), X_test.index)

In [34]:
test_referral_table = referral_table.loc[X_test.index]

In [38]:
def get_scores_per_window(x, y, group, threshold=0.5):
    corr = spearmanr(x, y)[0]
    mu_a = x.groupby(group).mean()
    mu_p = y.groupby(group).mean()
    mu_a_top = mu_a[mu_a.rank(ascending=False) / len(mu_a) < threshold]
    mu_p_top = mu_p[mu_p.rank(ascending=False) / len(mu_p) < threshold]
    overlap = mu_p_top.index.isin(mu_a_top.index).mean()
    return pd.Series([corr, overlap], index=['spearman', 'overlap'])

In [39]:
def evaluate_average_weekly_rank_correlation(test_referral_table, y_test, y_pred):
    grouped_y = test_referral_table.assign(y=y_test, pred=y_pred).set_index('referral_referraltakendate')\
        .groupby([pd.TimeGrouper('1W'), 'client_clientid'])['y'].mean()
    grouped_pred_y = test_referral_table.assign(y=y_test, pred=y_pred).set_index('referral_referraltakendate')\
        .groupby([pd.TimeGrouper('1W'), 'client_clientid'])['pred'].mean()
    time_grouped = pd.concat([grouped_y, grouped_pred_y], axis=1)
    return time_grouped.reset_index().groupby(['referral_referraltakendate']).\
        apply(lambda k: get_scores_per_window(k['y'], k['pred'], k['client_clientid'])).dropna().mean()

In [40]:
evaluate_average_weekly_rank_correlation(test_referral_table, y_test, y_pred)

/Users/davidsykes/.virtualenvs/twc/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  
/Users/davidsykes/.virtualenvs/twc/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/davidsykes/.virtualenvs/twc/lib/python3.5/site-packages/numpy/lib/function_base.py:3183: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/davidsykes/.virtualenvs/twc/lib/python3.5/site-packages/numpy/lib/function_base.py:3184: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/davidsykes/.virtualenvs/twc/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b

spearman    0.500569
overlap     0.665357
dtype: float64

In [42]:
referral_table[referral_table['client_clientid'].isnull()]

,referral_statusid,referral_referralonhold,referral_referraltakendate,referral_referralreadydate,referral_referralcollecteddate,referral_referralworkerid,referral_referralhandedworkerid,referral_clientid,referral_partnername,referral_partnerid,...,referral_referralnotes,referral_updatetimestamp,client_clientid,reference_date,futurereferraltargetfeature_futurereferralcount,futurereferraltargetfeature_futurereferralscore,futurereferraltargetfeature_futurereferralgaps,timefeature_referralnumber,timefeature_burstnumber,timefeature_totalreferralsforclient
referral_referralinstanceid,,,,,,,,,,,,,,,,,,,,,


# Try and recreate the same using the auto-retrain logic

In [23]:
from api.resources.retrain import train_model_from_json
from api.resources.retrain import construct_full_tables

In [63]:
def process_referral_table(referral_table):
    referral_table['referraltakendate'] = pd.to_datetime(referral_table['referraltakendate'])
    referral_table = referral_table.set_index('referralinstanceid')
    referral_table = referral_table.add_prefix('referral_')
    return referral_table

process_referral_table(tables['referral'])

,referral_addresslocalityid,referral_addresstypeid,referral_clientid,referral_dependantnumber,referral_ethnicityid,referral_livingwithpartner,referral_partnerid,referral_referralagencyid,referral_referralcollecteddate,referral_referralhandedworkerid,referral_referralpreparedworkerid,referral_referralreadydate,referral_referraltakendate,referral_referralworkerid,referral_statusid,referral_updatetimestamp
referralinstanceid,,,,,,,,,,,,,,,,
12,6,4,10,0,1,0,None,31,01/13/14 00:00:00,NaN,NaN,01/13/14 00:00:00,2014-01-13 00:00:00,4,3,04/01/14 09:53:43
13,6,1,10,0,1,0,None,86,02/03/14 00:00:00,NaN,NaN,02/03/14 00:00:00,2014-02-03 00:00:00,4,3,04/01/14 09:53:43
2631,6,1,10,0,1,0,None,32,06/09/14 15:31:00,7,7,06/09/14 15:31:00,2014-06-09 15:31:00,7,3,06/09/14 15:31:52
2733,6,1,10,0,1,0,None,86,06/16/14 15:46:00,7,7,06/16/14 15:46:00,2014-06-16 15:46:00,7,3,06/16/14 15:46:05
2873,6,1,10,0,1,0,None,86,06/23/14 16:42:00,7,7,06/23/14 16:42:00,2014-06-23 16:42:00,7,3,06/23/14 16:42:17
14956,6,4,10,0,1,0,None,65,10/18/16 13:06:00,26,26,10/18/16 13:06:00,2016-10-18 11:24:00,15,3,10/18/16 13:06:42
15293,NaN,4,10,0,1,0,None,89,11/08/16 16:46:00,1,1,11/08/16 16:46:00,2016-11-08 12:32:00,1,3,11/08/16 16:46:33
15739,NaN,2,10,0,1,0,None,136,12/02/16 15:29:00,26,26,12/02/16 15:29:00,2016-12-02 14:09:00,8,3,12/02/16 15:29:51
18,11,1,13,2,1,0,None,77,02/05/14 00:00:00,NaN,NaN,02/05/14 00:00:00,2014-02-05 00:00:00,4,3,04/01/14 09:54:22


In [61]:
referral_table[referral_table['referral_clientid']==3013]

,referral_statusid,referral_referralonhold,referral_referraltakendate,referral_referralreadydate,referral_referralcollecteddate,referral_referralworkerid,referral_referralhandedworkerid,referral_clientid,referral_partnername,referral_partnerid,...,referral_referralnotes,referral_updatetimestamp,client_clientid,reference_date,futurereferraltargetfeature_futurereferralcount,futurereferraltargetfeature_futurereferralscore,futurereferraltargetfeature_futurereferralgaps,timefeature_referralnumber,timefeature_burstnumber,timefeature_totalreferralsforclient
6126,3,0,2015-01-20 11:39:00,01/20/15 16:04:00,01/20/15 16:04:00,1,1.0,3013,None,NaN,...,<NAME> is sofa surfing and having to use her b...,01/20/15 16:04:23,3013.0,NaT,8.0,0.076712,4.0,1.0,1.0,11
6923,3,0,2015-03-13 12:05:00,03/13/15 17:35:00,03/13/15 17:35:00,6,13.0,3013,None,NaN,...,Came off JSA to do a job in Manchester but cou...,03/13/15 17:35:11,3013.0,NaT,9.0,0.115068,3.0,2.0,2.0,11
7250,3,0,2015-04-08 12:35:00,04/08/15 15:30:00,04/08/15 15:30:00,10,5.0,3013,None,NaN,...,Hannah has been homeless and had to flee from...,04/08/15 15:30:04,3013.0,NaT,8.0,0.095890,3.0,3.0,2.0,11
7370,3,0,2015-04-16 15:50:00,04/16/15 16:06:00,04/16/15 16:06:00,5,5.0,3013,None,NaN,...,New tenancy.,04/16/15 16:06:12,3013.0,NaT,7.0,0.076712,3.0,4.0,2.0,11
7447,3,0,2015-04-22 15:51:00,04/22/15 00:00:00,04/22/15 00:00:00,13,5.0,3013,None,NaN,...,"Hannah has recieved a letter of ""doubt"" regard...",04/23/15 10:38:24,3013.0,NaT,6.0,0.057534,3.0,5.0,2.0,11
7866,3,0,2015-05-26 15:59:00,05/26/15 16:44:00,05/26/15 16:44:00,5,1.0,3013,None,NaN,...,"Hannah has had no income since april, there wa...",05/26/15 16:44:35,3013.0,NaT,5.0,0.057534,2.0,6.0,3.0,11
8014,3,0,2015-06-09 15:20:00,06/09/15 17:08:00,06/09/15 17:08:00,1,1.0,3013,None,NaN,...,hannah was on a sanction and borrowed money of...,06/09/15 17:08:35,3013.0,NaT,4.0,0.038356,2.0,7.0,3.0,11
9427,3,0,2015-09-29 12:23:00,09/29/15 16:13:00,09/29/15 16:14:00,6,1.0,3013,None,NaN,...,Had to pay family debts which has left her wit...,09/29/15 16:14:00,3013.0,NaT,3.0,0.038356,1.0,8.0,4.0,11
10862,3,0,2016-01-04 12:06:00,01/04/16 16:41:00,01/04/16 16:41:00,1,1.0,3013,None,NaN,...,had a claim in for UC due to the fact she is p...,01/04/16 16:41:40,3013.0,NaT,2.0,0.038356,0.0,9.0,5.0,11
11271,3,0,2016-01-29 16:10:00,01/29/16 16:30:00,01/29/16 16:30:00,13,1.0,3013,None,NaN,...,Benefits have been stopped pending receipt of ...,01/29/16 16:30:44,3013.0,NaT,1.0,0.019178,0.0,10.0,5.0,11


In [26]:
tables = construct_full_tables(json_data)

In [31]:
json_data[1]

{'client': [{'addressline2': None,
   'addressline4': None,
   'addresslocalityid': 11,
   'addresspostcode': 'HD2 2',
   'addresssincedate': '06/12/14 00:00:00',
   'clientaddresstypeid': 1,
   'clientcountryid': None,
   'clientdateofbirth': '06/01/71 00:00:00',
   'clientethnicityid': 10,
   'clientid': 13,
   'clientismale': '0',
   'clientresidencyid': None,
   'dependantnumber': 2,
   'enabled': '1',
   'lastmodifieddate': '09/11/15 00:00:00',
   'lastmodifiedworkerid': 1,
   'livingwithpartner': '0',
   'partnerid': None,
   'updatetimestamp': '09/11/15 10:22:29'}],
 'clientissue': [],
 'referral': [{'addresslocalityid': 11,
   'addresstypeid': 1,
   'clientid': 13,
   'dependantnumber': 2,
   'ethnicityid': 1,
   'livingwithpartner': '0',
   'partnerid': None,
   'referralagencyid': 77,
   'referralcollecteddate': '02/05/14 00:00:00',
   'referralhandedworkerid': None,
   'referralinstanceid': 18,
   'referralpreparedworkerid': None,
   'referralreadydate': '02/05/14 00:00:00',

In [30]:
[t.columns for k,t in tables.items()]

[Index(['domesticcircumstancesid', 'referralinstanceid'], dtype='object'),
 Index(['clientissueid', 'referralinstanceid'], dtype='object'),
 Index(['clientid', 'clientissueid'], dtype='object'),
 Index(['referralinstanceid', 'referralreasonid'], dtype='object'),
 Index(['dietaryrequirementsid', 'referralinstanceid'], dtype='object'),
 Index(['benefittypeid', 'referralinstanceid'], dtype='object'),
 Index(['addressline2', 'addressline4', 'addresslocalityid', 'addresspostcode',
        'addresssincedate', 'clientaddresstypeid', 'clientcountryid',
        'clientdateofbirth', 'clientethnicityid', 'clientid', 'clientismale',
        'clientresidencyid', 'dependantnumber', 'enabled', 'lastmodifieddate',
        'lastmodifiedworkerid', 'livingwithpartner', 'partnerid',
        'updatetimestamp'],
       dtype='object'),
 Index(['referraldocumentid', 'referralinstanceid'], dtype='object'),
 Index(['addresslocalityid', 'addresstypeid', 'clientid', 'dependantnumber',
        'ethnicityid', 'liv

In [14]:
X, y, referral_table, model, message = train_model_from_json(json_tables)

TypeError: string indices must be integers

In [18]:
with open('../twc_api/api/requestNewData.json', 'r') as f:
    json_data = json.load(f)
# X, y, referral_table, model, message = train_model_from_json(json_data)

In [22]:
json_data[0]

{'client': [{'addressline2': None,
   'addressline4': None,
   'addresslocalityid': None,
   'addresspostcode': 'HD1 5',
   'addresssincedate': '12/02/16 00:00:00',
   'clientaddresstypeid': 2,
   'clientcountryid': None,
   'clientdateofbirth': '03/01/68 00:00:00',
   'clientethnicityid': 1,
   'clientid': 10,
   'clientismale': '1',
   'clientresidencyid': None,
   'dependantnumber': 0,
   'enabled': '1',
   'lastmodifieddate': '12/02/16 00:00:00',
   'lastmodifiedworkerid': 8,
   'livingwithpartner': '0',
   'partnerid': None,
   'updatetimestamp': '12/02/16 14:06:17'}],
 'clientissue': [{'clientid': 10, 'clientissueid': 9}],
 'referral': [{'addresslocalityid': 6,
   'addresstypeid': 4,
   'clientid': 10,
   'dependantnumber': 0,
   'ethnicityid': 1,
   'livingwithpartner': '0',
   'partnerid': None,
   'referralagencyid': 31,
   'referralcollecteddate': '01/13/14 00:00:00',
   'referralhandedworkerid': None,
   'referralinstanceid': 12,
   'referralpreparedworkerid': None,
   'refe

In [17]:
from collections import defaultdict
def construct_full_tables(json_data):
    tables = defaultdict(list)
    for row in json_data:
        for key in row:
            tables[key].append(pd.DataFrame.from_dict(row[key]))
    return {k: pd.concat(v) for k, v in tables.items()}

tables = construct_full_tables(json_data)

In [ ]:
tables['refe']

In [58]:
tables['client'].groupby('clientcountryid').size().sum()

641

In [92]:
client_old = training_data['client'].set_index('clientid')
client_new = tables['client'].set_index('clientid')

In [108]:
client_new['clientaddresstypeid'].value_counts(), client_old['clientaddresstypeid'].value_counts()

(1    2564
 2     719
 3     145
 4     104
 6     103
 7      85
 8      46
 5      45
 Name: clientaddresstypeid, dtype: int64, 1.0    3545
 2.0     838
 nan     576
 4.0     179
 6.0     168
 3.0     159
 7.0     116
 8.0      70
 5.0      40
 Name: clientaddresstypeid, dtype: int64)

In [105]:
tables['referraldocument']

""


In [103]:
old_nulls = client_old[client_old['clientcountryid'] == 'nan']

client_new.loc[old_nulls.index].groupby("clientcountryid").size()


/Users/davidsykes/.virtualenvs/twc/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  This is separate from the ipykernel package so we can avoid doing imports until


clientcountryid
2     1
14    2
24    1
dtype: int64

### Find columns that are missing

In [26]:
len(X_retrain.columns), len(X.columns)

(569, 593)

In [27]:
[c for c in X.columns if c not in X_retrain.columns]

['referraldietaryrequirements_1',
 'referraldietaryrequirements_2',
 'referraldietaryrequirements_3',
 'referraldietaryrequirements_4',
 'referraldietaryrequirements_5',
 'referraldietaryrequirements_6',
 'referraldietaryrequirements_7',
 'referraldietaryrequirements_8',
 'referraldietaryrequirements_9',
 'referraldietaryrequirements_10',
 'referraldietaryrequirements_11',
 'referraldietaryrequirements_12',
 'referraldocument_1',
 'referraldocument_2',
 'referraldocument_3',
 'referraldocument_4',
 'referraldocument_5',
 'client_clientethnicityid_1.0',
 'client_clientethnicityid_10.0',
 'client_clientethnicityid_11.0',
 'client_clientethnicityid_12.0',
 'client_clientethnicityid_13.0',
 'client_clientethnicityid_14.0',
 'client_clientethnicityid_15.0',
 'client_clientethnicityid_16.0',
 'client_clientethnicityid_17.0',
 'client_clientethnicityid_18.0',
 'client_clientethnicityid_2.0',
 'client_clientethnicityid_3.0',
 'client_clientethnicityid_4.0',
 'client_clientethnicityid_5.0',
 'c

In [30]:
y_pred = pd.Series(model.predict(X_test), X_test.index)

ValueError: Number of features of the model must match the input. Model n_features is 569 and input n_features is 593 

In [28]:
print(message)

Features Matrix generated consisting of 21181 referrals and 569 features
Train/Test sets split.
Train set: 16944 referrals.
Test set: 4237 referrals
Trained Model on: 16944 observations
Model Test Evaluation Metrics:
	Test Set Correlation of Predicted and Actual Mean Weekly Scores: 0.285111297280371
	Test Set Overlap of top 20.0% worst cases: 0.23163067552602426
Trained Model on: 21181 observations
